In [1]:
import time
from collections import deque, namedtuple

#import gym
import gymnasium as gym
import numpy as np
import PIL.Image
import tensorflow as tf
import utils
import random

from pyvirtualdisplay import Display
import torch
import torch.nn as nn

import public_tests
#from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Dense, Input
#from tensorflow.keras.losses import MSE
#from tensorflow.keras.optimizers import Adam

2023-10-31 13:31:04.734391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 13:31:05.612111: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
Display(visible=0, size=(840, 480)).start();

torch.manual_seed(42)

In [3]:
MEMORY_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate  
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps

In [4]:
print(MEMORY_SIZE)

100000


In [5]:
env = gym.make('LunarLander-v2')
env.reset()

state_size = env.observation_space.shape
num_actions = env.action_space.n

print('State Shape:', state_size)
print('Number of actions:', num_actions)

State Shape: (8,)
Number of actions: 4


In [6]:
# Select an action
action = 0

initial_state = env.reset()

# Run a single time step of the environment's dynamics with the given action.
next_state, reward, done, info, f = env.step(action)
#tuple = env.step(action)
print("The tuple")
print(tuple)

with np.printoptions(formatter={'float': '{:.3f}'.format}):
    print("Initial State:", initial_state)
    print("Action:", action)
    print("Next State:", next_state)
    print("Reward Received:", reward)
    print("Episode Terminated:", done)
    print("Info:", info)
    print("F :",f)

The tuple
<class 'tuple'>
Initial State: (array([0.007, 1.418, 0.746, 0.334, -0.009, -0.169, 0.000, 0.000],
      dtype=float32), {})
Action: 0
Next State: [0.015 1.425 0.745 0.308 -0.017 -0.167 0.000 0.000]
Reward Received: -0.42573787541755337
Episode Terminated: False
Info: False
F : {}


In [7]:
# # UNQ_C1
# # GRADED CELL

# # Create the Q-Network
# print("State size :",type(state_size))
# q_network = Sequential([
#     ### START CODE HERE ### 
#     Input(shape=state_size),
#     Dense(units=64,activation='relu'),
#     Dense(units=64,activation='relu'),
#     Dense(units=num_actions,activation='linear'),
    
#     ### END CODE HERE ### 
#     ])

# # Create the target Q^-Network
# target_q_network = Sequential([
#     ### START CODE HERE ### 
#     Input(shape=state_size),
#     Dense(units=64,activation='relu'),
#     Dense(units=64,activation='relu'),
#     Dense(units=num_actions,activation='linear'),
#     ### END CODE HERE ###
#     ])

# ### START CODE HERE ### 
# optimizer = Adam(learning_rate=ALPHA)
# ### END CODE HERE ###

# print(q_network.summary(expand_nested=True))

# print(target_q_network.summary())

# this is how you'd presumably create a dense nn on torch using the same
# layout as tf
q_network = nn.Sequential(
    nn.Linear(state_size[0], 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 4),
    
)

target_q_network = nn.Sequential(
    nn.Linear(state_size[0], 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 4),
    
)

optimizer = torch.optim.Adam(q_network.parameters(), lr=ALPHA)
optimizer = torch.optim.Adam(target_q_network.parameters(), lr=ALPHA)


In [8]:
experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
np.random.seed(1)
states = np.float32(np.random.rand(64, 8))
actions = np.float32(np.floor(np.random.uniform(0, 1, (64, )) * 4))
rewards = np.float32(np.random.rand(64, ))
next_states = np.float32(np.random.rand(64, 8))
done_vals = np.float32((np.random.uniform(0, 1, size=(64,)) > 0.96) * 1)

print(actions)


[0. 0. 2. 1. 2. 0. 2. 3. 3. 2. 3. 3. 2. 0. 0. 0. 1. 1. 3. 0. 3. 3. 3. 2.
 2. 1. 0. 1. 1. 2. 1. 1. 0. 0. 3. 1. 3. 0. 1. 2. 3. 1. 0. 1. 2. 2. 3. 0.
 0. 2. 3. 0. 2. 2. 1. 3. 2. 0. 1. 3. 3. 2. 2. 2.]


In [25]:
# def compute_loss(experiences, gamma, q_network, target_q_network):
#     """ 
#     Calculates the loss.
    
#     Args:
#       experiences: (tuple) tuple of ["state", "action", "reward", "next_state", "done"] namedtuples
#       gamma: (float) The discount factor.
#       q_network: (tf.keras.Sequential) Keras model for predicting the q_values
#       target_q_network: (tf.keras.Sequential) Karas model for predicting the targets
          
#     Returns:
#       loss: (TensorFlow Tensor(shape=(0,), dtype=int32)) the Mean-Squared Error between
#             the y targets and the Q(s,a) values.
#     """
    
#     # Unpack the mini-batch of experience tuples
#     states, actions, rewards, next_states, done_vals = experiences
    
#     # Compute max Q^(s,a)
#     max_qsa = tf.reduce_max(target_q_network(next_states), axis=0)

#     print(type(target_q_network(next_states)))
    
#     print(target_q_network(next_states))
#     print("numpy shape ",target_q_network(next_states).shape)
#     print("numpy max, " ,np.max(target_q_network(next_states),axis=0))
#     print("numpy dtype , ",target_q_network(next_states).dtype)
#     print(max_qsa)
    
#     # Set y = R if episode terminates, otherwise set y = R + γ max Q^(s,a).
#     ### START CODE HERE ### 
#     y = (rewards + (1-done_vals)*(gamma*max_qsa))
#     ### END CODE HERE ###
    
#     # Get the q_values
#     q_values = q_network(states)
#     q_values = tf.gather_nd(q_values, tf.stack([tf.range(q_values.shape[0]),
#                                                 tf.cast(actions, tf.int32)], axis=1))
        
#     # Compute the loss
#     ### START CODE HERE ### 
#     loss = MSE(y, q_values)
#     ### END CODE HERE ### 
    
#     return loss

def compute_loss(experiences, gamma, q_network, target_q_network):
    
    # Unpack the mini-batch of experience tuples
    
    states, actions, rewards, next_states, done_vals = experiences
    print(type(states))
    print(type(actions))
    print(experiences)
    #input_tensor = torch.randn(20,20)
    #print(type(input_tensor))
    #print(input_tensor)
    print(type(states))
    print(type(rewards))
    #print("END")
    n_statae = target_q_network(next_states)

    #print(n_statae)
    max_qsa , indicies  = torch.max(torch.from_numpy(n_statae), dim=1)
    
    #print("Actions :",actions)

    #print("max_qsa ", max_qsa)
    y = (rewards + (1-done_vals)*(gamma*max_qsa.numpy()))
    
    q_values = q_network(states)


    

    #print("q_values ,",q_values)
    #print("action shape ",type(actions))
    #print(np.arange(actions.shape[0]))
    #print("q_values 64, ",q_values[np.arange(actions.shape[0]),actions.astype(int)])
    q_values = q_values[np.arange(actions.shape[0]),actions.astype(int)]
    #print("Y ,",y.shape)
    #print("q_values, ",q_values.size())
    mseloss = torch.nn.MSELoss()
    loss = mseloss(torch.from_numpy(y),torch.from_numpy(q_values))
    #mse_loss = torch.nn.functional.mse_loss(y,q_values)

    #print(loss)
    return loss

public_tests.test_compute_loss(compute_loss)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([[4.17021990e-01, 7.20324516e-01, 1.14374816e-04, 3.02332580e-01,
        1.46755889e-01, 9.23385918e-02, 1.86260208e-01, 3.45560730e-01],
       [3.96767467e-01, 5.38816750e-01, 4.19194520e-01, 6.85219526e-01,
        2.04452246e-01, 8.78117442e-01, 2.73875929e-02, 6.70467496e-01],
       [4.17304814e-01, 5.58689833e-01, 1.40386939e-01, 1.98101491e-01,
        8.00744593e-01, 9.68261600e-01, 3.13424170e-01, 6.92322612e-01],
       [8.76389146e-01, 8.94606650e-01, 8.50442126e-02, 3.90547849e-02,
        1.69830427e-01, 8.78142476e-01, 9.83468369e-02, 4.21107620e-01],
       [9.57889557e-01, 5.33165276e-01, 6.91877127e-01, 3.15515637e-01,
        6.86500907e-01, 8.34625661e-01, 1.82882771e-02, 7.50144303e-01],
       [9.88861084e-01, 7.48165667e-01, 2.80443996e-01, 7.89279342e-01,
        1.03226006e-01, 4.47893530e-01, 9.08595502e-01, 2.93614149e-01],
       [2.87775338e-01, 1.30028576e-01, 1.93669572e-02, 6.78835511e-01,
        2

In [10]:
print("hello")

hello


In [40]:
start = time.time()

SEED = 0              # seed for pseudo-random number generator
MINIBATCH_SIZE = 64   # mini-batch size
TAU = 1e-3            # soft update parameter
E_DECAY = 0.995       # ε decay rate for ε-greedy policy
E_MIN = 0.01          # minimum ε value for ε-greedy policy

num_episodes = 2000
max_num_timesteps = 1000

total_point_history = []

num_p_av = 100    # number of total points to use for averaging
epsilon = 1.0     # initial ε value for ε-greedy policy

# Create a memory buffer D with capacity N
memory_buffer = deque(maxlen=MEMORY_SIZE)

weights = q_network.state_dict()
target_q_network.load_state_dict(weights)

for source_param, target_param in zip(q_network.parameters(), target_q_network.parameters()):
    assert torch.all(torch.eq(source_param, target_param))
#print(weights)

for i in range(num_episodes):
    state = env.reset()
    total_points = 0
    
    for j in range(max_num_timesteps):
    
        actions = q_network(torch.from_numpy(state[0]))

        #print("Theser are the actions: ",actions)
        #print(actions)
        #print(hello)
        #e-greedy policy
        if random.random() > epsilon:
            new_action=torch.argmax(actions)
            
            
        else:
            new_action=torch.argmax(actions)
            

        epsilon = max(E_MIN, E_DECAY*epsilon)
        new_states,reward,done,_,_ = env.step(new_action.item())
        
        memory_buffer.append(experience(state,new_action,reward,new_states,done))

       
      
       

        if (j + 1)%4 == 0 and len(memory_buffer) > 64 :
            experiences = random.sample(memory_buffer, k=64)

            #states = tf.convert_to_tensor(np.array([e.state for e in experiences if e is not None]),dtype=tf.float32)
            print([e.action for e in experiences])
            #print(experiences)
            print(test_states)
            print("yoooooo we here aya")
            #print(np.array([e.state for e in experiences if e is not None]))
            #print(experiences)
            print("yoooo this aint it")

            
            
            

            break
    break
        
        

[tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1)]
<generator object <genexpr> at 0x7f4ebf333d10>
yoooooo we here aya
yoooo this aint it


In [157]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])

# find the maximum value and index of the tensor along the second dimension
max_val, max_idx = torch.max(x, dim=1)

print(x)
print(max_val)
print(max_idx)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([3, 6])
tensor([2, 2])


In [163]:
counter = 0

for i in range(max_num_timesteps):
    counter = counter + 1
    if (i + 1) % 4 == 0:
        print(counter)
    

4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
508
512
516
520
524
528
532
536
540
544
548
552
556
560
564
568
572
576
580
584
588
592
596
600
604
608
612
616
620
624
628
632
636
640
644
648
652
656
660
664
668
672
676
680
684
688
692
696
700
704
708
712
716
720
724
728
732
736
740
744
748
752
756
760
764
768
772
776
780
784
788
792
796
800
804
808
812
816
820
824
828
832
836
840
844
848
852
856
860
864
868
872
876
880
884
888
892
896
900
904
908
912
916
920
924
928
932
936
940
944
948
952
956
960
964
968
972
976
980
984
988
992
996
1000
